## Flux comparions
This notebook compares the predictions of the R. opacus PD630 GSM with the fluxes from 13C-metabolic flux analysis (13C-MFA)

### Methods: 
<ol>
<li>Calculate FBA and pFBA growth rates for each of the experimental conditions</li>
<li>Calculate EFLUX2 growth rates for each of the experimental conditions</li>
</ol>

### Imports

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import cobra
import pandas as pd
import numpy as np
from edd_utils import login, export_study, export_metadata

### Load model 

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07fb5a46f1650
Number of metabolites,1583
Number of reactions,2385
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [3]:
# edd_study_slug = 'input-test-study'
# edd_server = 'public-edd.agilebiofoundry.org'
# user= 'garrettroell'
# session = login(edd_server=edd_server, user=user)
# # df = utils.load_study(edd_study_slug=edd_study_slug,edd_server=edd_server,
# #   default_to_system_user=False)

In [ ]:
# df = export_study(session, edd_study_slug, edd_server=edd_server)
# df

### Get 13C MFA measured fluxes for phenol

In [5]:
glucose_fluxes = pd.read_csv('../13C_flux_data/13C_glucose_flux_data.csv')
glucose_fluxes

,Pathway,Reaction ID,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound,Unnamed: 7,Unnamed: 8
0,Glucose Uptake,NaN,NaN,Gluc.ext + ATP -> G6P,1.000000e+02,100.0000,1.000000e+02,NaN,NaN
1,Glycolysis,PGI,False,G6P <-> F6P,-1.610000e+00,-2.0900,1.420000e+00,NaN,NaN
2,NaN,NaN,NaN,F6P + ATP -> FBP,0.000000e+00,0.0000,1.910000e+00,NaN,NaN
3,NaN,FBA,False,FBP <-> DHAP + GAP,0.000000e+00,0.0000,1.910000e+00,NaN,NaN
4,NaN,TPI,False,DHAP <-> GAP,0.000000e+00,0.0000,1.910000e+00,NaN,NaN
5,NaN,NaN,NaN,GAP <-> 3PG + ATP + NADH,8.656000e+01,82.6100,8.884000e+01,NaN,NaN
6,NaN,NaN,NaN,3PG <-> PEP,7.852000e+01,71.6500,8.126000e+01,NaN,NaN
7,NaN,NaN,NaN,PEP <-> Pyr + ATP,1.902000e+01,-10.8900,7.515000e+01,NaN,NaN
8,Pentose Phosphate Pathway,NaN,NaN,G6P <-> 6PG + NADPH,1.000000e+02,96.9600,1.000000e+02,NaN,NaN
9,NaN,NaN,NaN,6PG <-> Ru5P + CO2 + NADPH,6.620000e+00,2.0700,1.092000e+01,NaN,NaN


### Run glucose flux FBA

In [ ]:
model.objective = "Growth"
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    model.medium = medium
    solution = model.optimize()
    print(solution)

### Get 13C MFA measured fluxes

In [ ]:
glucose_flux_path = "../13C_flux_data/13C_glucose_flux_data.csv"
measured_glucose_df = pd.read_csv(glucose_flux_path)
measured_glucose_df

Eliminate all rows that do not have a defined reactionID

In [ ]:
measured_glucose_df = measured_glucose_df[~pd.isnull(measured_glucose_df['Reaction ID'])]
measured_glucose_df

In [ ]:
measured_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
#     print(measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0])
    measured_flux = measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Flux'].values[0] / 10
#     if measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0]:
#         measured_flux = -1 * measured_flux
    measured_flux_dict[reaction_id] = measured_flux
measured_flux_dict

### Get FBA simulated fluxes

In [ ]:
simulated_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    simulated_flux_dict[reaction_id] = solution.fluxes[reaction_id]

simulated_flux_dict

### Plot FBA vs measured fluxes

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(simulated_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
# ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".3")
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'FBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with pfba

In [ ]:
pfba_solution = cobra.flux_analysis.pfba(model)
len(pfba_solution.fluxes)

In [ ]:
pfba_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    pfba_flux_dict[reaction_id] = pfba_solution.fluxes[reaction_id]

pfba_flux_dict

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(pfba_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'pFBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with geometric FBA

In [ ]:
# geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
# len(geometric_fba_sol.fluxes)

In [ ]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(r.name, r)